In [44]:
from collections import defaultdict
import heapq
import time
import requests
import random
import csv

In [46]:
class LikeSystem:
    def __init__(self):
        self.user_likes = defaultdict(set)  #user_id = set of post ids
        self.post_likes = defaultdict(int)  #post_id = like count
        self.timestamp_likes = defaultdict(list)  #post_id = list of timestamps
        self.like_history = []   #Stack of (user_id, post_id, timestamp)


    def like(self, user_id, post_id):
        if post_id not in self.user_likes[user_id]:
            self.user_likes[user_id].add(post_id)
            self.post_likes[post_id]+=1
            ts = time.time()
            self.timestamp_likes[post_id].append(ts)
            self.like_history.append((user_id,post_id,ts))
            print(f"{user_id} liked {post_id}")
        else:
            print(f"{user_id} already liked {post_id}")


    def undo_like(self):
        if not self.like_history:
            print("No likes to undo.")
            return
        user_id, post_id, ts = self.like_history.pop()
        self.user_likes[user_id].remove(post_id)
        self.post_likes[post_id] -=1
        self.timestamp_likes[post_id].remove(ts)
        print(f"{user_id} unliked {post_id}")


    def get_top_liked_posts(self, n=3):
        top_posts = heapq.nlargest(n, self.post_likes.items(), key =lambda x:x[1])
        print(f"Top {n} posts by likes:")
        for post_id, count in top_posts:
            print(f"Post {post_id} -> {count} likes")
        return top_posts


    def get_recent_trending_posts(self, hours =24):
        now = time.time()
        cutoff = now - (hours *3600)
        post_trending = []

        for post_id, timestamps in self.timestamp_likes.items():
            recent_likes = [ts for ts in timestamps if ts >= cutoff]
            if recent_likes:
                post_trending.append((post_id, len(recent_likes)))
                                     
        trending_sorted = sorted(post_trending, key = lambda x :x[1], reverse =True)
        print(f"Trending posts in last {hours} hours:")
        for post_id, count in trending_sorted:
            print(f"Post {post_id} -> {count} recent likes")
        return trending_sorted



    def get_user_likes(self, user_id):
        liked_posts = list(self.user_likes.get(user_id, []))
        print(f"\n{user_id} has liked: {liked_posts}")
        return liked_posts
#Get Likes on a Post Between Two Timestamps
    def get_likes_between(self, post_id, start_ts, end_ts):
        timestamps = self.timestamp_likes.get(post_id,[])
        filtered = [ts for ts in timestamps if start_ts<= ts<= end_ts]
        print(f"\nPost {post_id} got {len(filtered)} likes between given time range.")
        return filtered
#Get Top Users Who Like the Most
    def get_top_users(self, n=3):
        user_like_counts = {user: len(posts) for user, posts in self.user_likes.items()}
        top_users = heapq.nlargest(n, user_like_counts.items(), key = lambda x: x[1])
        print(f"\nTop {n} users by total likes given :")
        for user, count in top_users:
            print(f"{user} -> {count} likes given")
        return top_users



#Export Current Data to CSV
    def export_data_to_csv(self, filename ="likedata.csv"):
        with open(filename, "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["user_id", "post_id", "timestamp"])
            for user_id, post_id, ts in self.like_history:
                writer.writerow([user_id, post_id, ts])
        print(f"\nExported like history to {filename}")
        

#Most Active Users in Recent Time (e.g., Last 3 Hours)
    def get_most_active_users_recently(self, hours =3):
        now = time.time()
        cutoff = now - (hours*3600)
        user_counter = defaultdict(int)

        for user_id , post_id, ts in self.like_history:
            if ts>= cutoff:
                user_counter[user_id]+=1

        sorted_users = sorted(user_counter.items(), key = lambda x:x[1], reverse =True)
        print(f"\nMost active users in the last {hours} hours:")
        for user, count in sorted_users:
            print(f"{user} -> {count} recent likes")
        return sorted_users

In [48]:
#Fetch fake users
users = requests.get("https://jsonplaceholder.typicode.com/users").json()

#Fetch fake posts
posts = requests.get("https://jsonplaceholder.typicode.com/posts").json()


# Extract User IDs and Post IDs
user_ids = [str(user['id']) for user in users] # IDs as strings
post_ids = [f"post{post['id']}" for post in posts] #formatted post IDs like 'post1'

print(f"Fetched {len(user_ids)} users and {len(post_ids)} posts.")

Fetched 10 users and 100 posts.


In [50]:

#Intialize LikeSytem
ls =LikeSystem()

#Simulate 100 random likes

for _ in range(100):
    user = random.choice(user_ids)
    post = random.choice(post_ids)
    ls.like(user,post)



10 liked post33
2 liked post9
8 liked post52
2 liked post62
10 liked post15
3 liked post50
2 liked post11
9 liked post41
6 liked post26
9 liked post4
9 liked post33
9 liked post69
8 liked post49
7 liked post3
6 liked post23
6 liked post59
3 liked post16
9 liked post79
4 liked post26
4 liked post71
8 liked post17
7 liked post33
10 liked post61
6 liked post76
7 liked post78
1 liked post20
6 liked post40
7 liked post54
3 liked post32
10 liked post20
8 liked post26
9 liked post5
10 liked post53
1 liked post86
1 liked post87
9 liked post13
8 liked post70
2 liked post85
2 liked post45
7 liked post19
9 liked post57
2 liked post16
9 liked post14
2 liked post100
2 liked post38
3 liked post77
1 liked post72
10 liked post55
1 liked post26
4 liked post76
4 liked post49
10 liked post35
8 liked post36
9 liked post97
7 liked post16
5 liked post84
3 liked post85
4 liked post37
7 liked post23
10 liked post12
1 liked post43
10 liked post66
3 liked post27
1 liked post19
8 liked post86
10 liked post73
3 l

In [36]:
ls.undo_like()

5 unliked post66


In [38]:
ls.get_top_liked_posts(5)

Top 5 posts by likes:
Post post48 -> 4 likes
Post post73 -> 3 likes
Post post83 -> 3 likes
Post post3 -> 3 likes
Post post92 -> 2 likes


[('post48', 4), ('post73', 3), ('post83', 3), ('post3', 3), ('post92', 2)]

In [40]:
ls.get_recent_trending_posts(12)

Trending posts in last 12 hours:
Post post48 -> 4 recent likes
Post post73 -> 3 recent likes
Post post83 -> 3 recent likes
Post post3 -> 3 recent likes
Post post92 -> 2 recent likes
Post post10 -> 2 recent likes
Post post41 -> 2 recent likes
Post post1 -> 2 recent likes
Post post15 -> 2 recent likes
Post post4 -> 2 recent likes
Post post2 -> 2 recent likes
Post post25 -> 2 recent likes
Post post59 -> 2 recent likes
Post post70 -> 2 recent likes
Post post35 -> 2 recent likes
Post post79 -> 2 recent likes
Post post54 -> 2 recent likes
Post post44 -> 2 recent likes
Post post6 -> 2 recent likes
Post post31 -> 2 recent likes
Post post20 -> 2 recent likes
Post post65 -> 1 recent likes
Post post7 -> 1 recent likes
Post post11 -> 1 recent likes
Post post9 -> 1 recent likes
Post post60 -> 1 recent likes
Post post71 -> 1 recent likes
Post post27 -> 1 recent likes
Post post52 -> 1 recent likes
Post post22 -> 1 recent likes
Post post32 -> 1 recent likes
Post post96 -> 1 recent likes
Post post78 ->

[('post48', 4),
 ('post73', 3),
 ('post83', 3),
 ('post3', 3),
 ('post92', 2),
 ('post10', 2),
 ('post41', 2),
 ('post1', 2),
 ('post15', 2),
 ('post4', 2),
 ('post2', 2),
 ('post25', 2),
 ('post59', 2),
 ('post70', 2),
 ('post35', 2),
 ('post79', 2),
 ('post54', 2),
 ('post44', 2),
 ('post6', 2),
 ('post31', 2),
 ('post20', 2),
 ('post65', 1),
 ('post7', 1),
 ('post11', 1),
 ('post9', 1),
 ('post60', 1),
 ('post71', 1),
 ('post27', 1),
 ('post52', 1),
 ('post22', 1),
 ('post32', 1),
 ('post96', 1),
 ('post78', 1),
 ('post69', 1),
 ('post53', 1),
 ('post63', 1),
 ('post87', 1),
 ('post93', 1),
 ('post81', 1),
 ('post36', 1),
 ('post46', 1),
 ('post34', 1),
 ('post95', 1),
 ('post77', 1),
 ('post40', 1),
 ('post5', 1),
 ('post85', 1),
 ('post42', 1),
 ('post12', 1),
 ('post17', 1),
 ('post88', 1),
 ('post82', 1),
 ('post14', 1),
 ('post55', 1),
 ('post72', 1),
 ('post56', 1),
 ('post99', 1),
 ('post76', 1),
 ('post37', 1),
 ('post29', 1),
 ('post66', 1),
 ('post21', 1),
 ('post58', 1),


In [52]:
ls.get_user_likes("1")
ls.get_likes_between("post15", time.time() - 3600, time.time())
ls.get_top_users(5)
ls.get_most_active_users_recently(1)
ls.export_data_to_csv("output.csv")


1 has liked: ['post86', 'post72', 'post20', 'post43', 'post19', 'post81', 'post26', 'post87', 'post14']

Post post15 got 2 likes between given time range.

Top 5 users by total likes given :
9 -> 14 likes given
3 -> 13 likes given
7 -> 13 likes given
2 -> 11 likes given
10 -> 10 likes given

Most active users in the last 1 hours:
9 -> 14 recent likes
3 -> 13 recent likes
7 -> 13 recent likes
2 -> 11 recent likes
10 -> 10 recent likes
8 -> 9 recent likes
6 -> 9 recent likes
1 -> 9 recent likes
4 -> 8 recent likes
5 -> 2 recent likes

Exported like history to output.csv
